# OpenAI Usage

## Description 
Input and process sentences using OpenAI's API

In [1]:
import json
import os
import openai
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env

True

In [3]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
output_format = {
    "sent_text": "string",
    "sent_person": "string",
    "sent_summary": "string",
    "sent_label": "string",
    "sent_categories": [{
        "category": "int",
        "reason": "string"
    }],
    "verb_phrase": "string",
    "tense": "string",
    "verb_is_stative": "bool",
    "verb_voice": "string",
    "verb_sentiment": "string",
    "verb_categories": [{
        "category": "int",
        "reason": "string"
    }],
    "subject_text": "string",
    "subject_is_plural": "boolean",
    "subject_semantics": "string",
    "object_text": "string",
    "object_is_plural": "boolean",
    "object_semantics": "string",
    "errors": "boolean"
}
prompt1 = 'You are a linguist and NLP expert. ' \
    'Sentence: sent_text ' \
    'For the Sentence provided, indicate if it is in the first, second or third person, ' \
    'and if there are grammatical or spelling errors in it. Also, provide a summarization of the ' \
    'Sentence, and create a short label for it in 30 characters or less. ' \
    'Indicate the possible semantic categories of the Sentence based on ALL the words in the Sentence, ' \
    'using the numbers of at least 2 of the categories listed below. ' \
    'Explain the reasoning for the categorizations. ' \
    'Indicate the main verb phrase and the verb tense, whether the verb is active or stative, and ' \
    'if the verb is in an active or passive voice. ' \
    'Make sure to capture the full text of phrasal verbs such as "lock up" or "talk down to", ' \
    'and the sentiment of the verb phrase - whether positive, negative or neutral. ' \
    'Indicate the possible meanings of the verb phrase based on the explicit Sentence text, ' \
    'using the numbers of at least 2 semantic categories. Explain the reasoning for the categorizations. '
prompt2 = 'The semantic categories are: ' \
    '1. acquisition such as by purchase or sale, gift-giving, donation, finding something, capture, seizure, ' \
    'robbery and theft ' \
    '2. aggressive, hostile or criminal act such as attack, assault, coercion, intimidation, bribery, threatening, ' \
    'deception, dishonesty, betrayal, resistance, murder, homicide, invasion, occupation ' \
    '3. agreement such as coming to an agreement, agreeing on something, and signing a contract ' \
    '4. any type of agricultural, apiculture, viniculture and aquacultural act or event ' \
    '5. any type of art and entertainment act or event including attending a movie, visiting a museum, ' \
    'playing a game, and hiking ' \
    '6. any attempt to do or achieve something ' \
    '7. avoidance such as bans, boycotts, escape, evasion, ignoring, overlooking, prevention and concealment ' \
    '8. bodily act such as movement, eating, drinking, grooming and sleeping ' \
    '9. change such as increase, decrease or physical change such as melting, bending and vaporization ' \
    '10. any type of cognition such as thinking, focusing, reading, assessing, characterizing, believing, making a ' \
    'decision, remembering or forgetting, having a goal, learning, having knowledge or skills, and understanding ' \
    '11. any type of emotion or feeling ' \
    '12. any type of sensory perception including pain and hunger ' \
    '13. any type of communication, communicating and speech acts such as recommending, acknowledging, denying, ' \
    'promising, making a request, boasting, joking, greeting, asking a question, granting permission, refusing, ' \
    'rejecting, deriding, slandering or surrendering ' \
    '14. competition or competing including playing a game ' \
    '15. continuation ' \
    '16. delay or wait' \
    '17. disagreement or dispute ' \
    '18. distribution or supply of something ' \
    '19. start of something' \
    '20. end of something, some event, or someone ' \
    '21. success ' \
    '22. failure ' \
    '23. win or loss ' \
    '24. any type of environmental or ecological event such as a disaster, weather event, or natural phenomenon ' \
    '25. any type of economic event such as a recesssion or inflation ' \
    '26. any type of financial event such as depositing or withdrawing money, releasing an annual report or ' \
    'paying taxes ' \
    '27. health-related event or act such as contracting a disease, addiction, a physical injury, frailty, ' \
    'having an allergic reaction, getting a vaccine, malnutrition and sterility ' \
    '28. impact and collision ' \
    '29. inclusion, unification and attachment such as adding to a list and assembling something ' \
    '30. information and data handling including IT operations ' \
    '31. issuing and publishing such as a newspaper, magazine or press release ' \
    '32. any legal or judicial event such as a trial, verdict, jury selection and judicial ruling ' \
    '33. life event such as birth, death, marriage and divorce ' \
    '34. measurement, measuring and reported assessment, count, percentages, etc. ' \
    '35. meeting and encounter such as party, chance encounter or ceremony ' \
    '36. any type of movement, travel or transportation such as entering or leaving a port or train station, ' \
    'traveling on a road, loading a truck or rail car, or making incremental changes such as pouring liquid from ' \
    'one container to another ' \
    '37. any political occurrence such as an election, coup or transfer of power, and political campaign ' \
    '38. any type of production, manufacture and creation event such as designing, building or producing ' \
    'products in a factory ' \
    '39. punishment ' \
    '40. reward and compensation ' \
    '41. removal and restriction including removal of raw materials or a siege ' \
    '42. return, recovery and release ' \
    '43. risk taking including gambling ' \
    '44. search ' \
    '45. separation ' \
    '46. any type of space event such as a meteor shower, sun spots, eclipse, or rocket or satellite launch ' \
    '47. any type of storage event such as moving cargo into or from storage, stockpiling or hoarding ' \
    '48. substitution of something or someone for something or someone else ' \
    '49. utilization and consumption ' \
    '50. violation of agreement ' \
    '51. possession ' \
    '52. description of a characteristic or an attribute of a person, place, event, or thing such as its shape, ' \
    'temperature, color, weight, population, level of pollution, and percentage '
prompt3 = \
    'Determine all the sentence subjects and objects, if subjects and/or objects are present. ' \
    'Indicate if any of the subjects and/or objects are singular or plural, and indicate their semantics as ' \
    '"person", "government", "organization, sub-organization or group of persons", "animal", ' \
    '"plant", "place", "environment or condition", inanimate object or substance", "historical event", ' \
    '"law, policy or legal decision", "quantity or measurement", or "other". '
prompt4 = f'Return the response as a JSON object with a shape of {output_format} '

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         prompt1.replace("sent_text", "Liz Cheney was beaten by Harriet Hageman in the WY primary.")
         +prompt2+prompt3+prompt4}
    ],
    temperature=0.3
)
print(response)

{
  "id": "chatcmpl-7qmxO29SdxsNXZPVmOTy0hM6hNloY",
  "object": "chat.completion",
  "created": 1692815602,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{'sent_text': 'Liz Cheney was beaten by Harriet Hageman in the WY primary.',\n 'sent_person': 'third',\n 'sent_summary': 'Liz Cheney lost to Harriet Hageman in the Wyoming primary.',\n 'sent_label': 'Primary Election Result',\n 'sent_categories': [{'category': 37, 'reason': 'The sentence describes a political occurrence, specifically a primary election result.'}],\n 'verb_phrase': 'was beaten',\n 'tense': 'past',\n 'verb_is_stative': False,\n 'verb_voice': 'passive',\n 'verb_sentiment': 'negative',\n 'verb_categories': [{'category': 23, 'reason': 'The verb phrase \"was beaten\" indicates a loss in the election.'}],\n 'subject_text': 'Liz Cheney',\n 'subject_is_plural': False,\n 'subject_semantics': 'person',\n 'object_text': 'Harriet Hageman'

In [4]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         prompt1.replace("sent_text", "The economy was sluggish.")+prompt2+prompt3+prompt4}
    ],
    temperature=0
)
print(response)

{
  "id": "chatcmpl-7qmy6khsdGjCToRX4PVokE7mHmLHD",
  "object": "chat.completion",
  "created": 1692815646,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{'sent_text': 'The economy was sluggish.',\n 'sent_person': 'third',\n 'sent_summary': 'The economy was slow.',\n 'sent_label': 'Economy Sluggish',\n 'sent_categories': [{'category': 25, 'reason': 'The sentence is about the state of the economy.'}],\n 'verb_phrase': 'was sluggish',\n 'tense': 'past',\n 'verb_is_stative': True,\n 'verb_voice': 'active',\n 'verb_sentiment': 'negative',\n 'verb_categories': [{'category': 9, 'reason': 'The verb phrase describes a state of the economy.'}],\n 'subject_text': 'The economy',\n 'subject_is_plural': False,\n 'subject_semantics': 'environment or condition',\n 'object_text': '',\n 'object_is_plural': False,\n 'object_semantics': '',\n 'errors': False}"
      },
      "finish_reason": "stop"
    }
  ],
  

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         prompt1.replace("sent_text", "The judge threw the book at the criminal.")+prompt2+prompt3+prompt4}
    ],
    temperature=0.3
)
print(response)

{
  "id": "chatcmpl-7qmyWkXrmBOtv24HnH35mqMni4fFq",
  "object": "chat.completion",
  "created": 1692815672,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"The judge threw the book at the criminal.\",\n  \"sent_person\": \"third\",\n  \"sent_summary\": \"The judge punished the criminal severely.\",\n  \"sent_label\": \"Judge punishes criminal\",\n  \"sent_categories\": [\n    {\n      \"category\": 32,\n      \"reason\": \"The sentence mentions a judge and a criminal, indicating a legal or judicial event.\"\n    },\n    {\n      \"category\": 39,\n      \"reason\": \"The sentence describes the judge's action of punishing the criminal.\"\n    }\n  ],\n  \"verb_phrase\": \"threw the book at\",\n  \"tense\": \"past\",\n  \"verb_is_stative\": false,\n  \"verb_voice\": \"active\",\n  \"verb_sentiment\": \"negative\",\n  \"verb_categories\": [\n    {\n      \"category\": 39,\n   

In [7]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         prompt1.replace("sent_text", "The weather is warm.")+prompt2+prompt3+prompt4}
    ],
    temperature=0
)
print(response)

{
  "id": "chatcmpl-7qmziHc9SkT9b4S4WgqziU32x9Hck",
  "object": "chat.completion",
  "created": 1692815746,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{'sent_text': 'The weather is warm.',\n 'sent_person': 'third',\n 'sent_summary': 'The weather is warm.',\n 'sent_label': 'Warm weather',\n 'sent_categories': [{'category': 23, 'reason': 'The sentence describes the state of the weather.'}],\n 'verb_phrase': 'is warm',\n 'tense': 'present',\n 'verb_is_stative': True,\n 'verb_voice': 'active',\n 'verb_sentiment': 'positive',\n 'verb_categories': [{'category': 9, 'reason': 'The verb phrase describes a physical change in temperature.'}],\n 'subject_text': 'The weather',\n 'subject_is_plural': False,\n 'subject_semantics': 'environment or condition',\n 'object_text': '',\n 'object_is_plural': False,\n 'object_semantics': '',\n 'errors': False}"
      },
      "finish_reason": "stop"
    }
  ],
  "

In [8]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         prompt1.replace("sent_text", "Mary is prety.")+prompt2+prompt3+prompt4}
    ],
    temperature=0
)
print(response)

{
  "id": "chatcmpl-7qn0CdowOUGxjd391PV0KWYGozDL4",
  "object": "chat.completion",
  "created": 1692815776,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"Mary is prety.\",\n  \"sent_person\": \"third\",\n  \"sent_summary\": \"Mary is pretty.\",\n  \"sent_label\": \"Mary is pretty\",\n  \"sent_categories\": [\n    {\n      \"category\": 11,\n      \"reason\": \"The sentence describes a feeling or emotion.\"\n    },\n    {\n      \"category\": 52,\n      \"reason\": \"The sentence describes a characteristic of a person.\"\n    }\n  ],\n  \"verb_phrase\": \"is pretty\",\n  \"tense\": \"present\",\n  \"verb_is_stative\": true,\n  \"verb_voice\": \"active\",\n  \"verb_sentiment\": \"positive\",\n  \"verb_categories\": [\n    {\n      \"category\": 11,\n      \"reason\": \"The verb phrase expresses a positive feeling or emotion.\"\n    },\n    {\n      \"category\": 52,\n      